In [ ]:
#import pandas as pd
!wget http://www-public.tem-tsp.eu/~zhang_da/pub/dataset_tsmc2014.zip
!unzip dataset_tsmc2014.zip

--2024-05-29 13:44:41--  http://www-public.tem-tsp.eu/~zhang_da/pub/dataset_tsmc2014.zip
Resolving www-public.tem-tsp.eu (www-public.tem-tsp.eu)... 157.159.10.107, 2001:660:3203:100:1:0:80:107
Connecting to www-public.tem-tsp.eu (www-public.tem-tsp.eu)|157.159.10.107|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25546284 (24M) [application/zip]
Saving to: ‘dataset_tsmc2014.zip.8’

dataset_tsmc2014.zi 100%[===================>]  24.36M  7.15MB/s    in 4.6s    

2024-05-29 13:44:46 (5.31 MB/s) - ‘dataset_tsmc2014.zip.8’ saved [25546284/25546284]

Archive:  dataset_tsmc2014.zip
replace dataset_tsmc2014/dataset_TSMC2014_NYC.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import pandas as pd
columns = ['User ID',
 'Venue ID',
 'Venue category ID',
 'Venue category name',
 'Latitude',
 'Longitude',
 'Timezone',
 'UTC time']

In [ ]:
NUMCHECKIN = 7000

df = pd.read_csv('dataset_tsmc2014/dataset_TSMC2014_NYC.txt', sep='\t', encoding='latin-1', names=columns)
df = df.sample(NUMCHECKIN)
print(df)

In [ ]:

checkin_counts = df['User ID'].value_counts() # Conta il numero di check-in per ogni utente
users_with_sufficient_checkins = checkin_counts[checkin_counts >= 5].index   # Filtra gli utenti con meno di 5 check-in
df = df[df['User ID'].isin(users_with_sufficient_checkins)]  # Mantieni solo le righe relative agli utenti con almeno 5 check-in

print(f"Numero di utenti con meno di 5 check-in: {len(checkin_counts[checkin_counts < 5])}")

In [ ]:
from sklearn.preprocessing import LabelEncoder


le = LabelEncoder()
df['Venue ID'] = le.fit_transform(df['Venue ID'])
df['Venue category ID'] = le.fit_transform(df['Venue category ID'])
df.drop(['Venue category name'],axis=1,inplace=True)
print(df)

In [ ]:
import math

def distance_between_coordinates(lat1, lon1, lat2, lon2):
    radius = 6371  # Radius of the Earth in km

    # Convert latitude and longitude from degrees to radians
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    # Calculate the differences in latitude and longitude<
    d_lat = lat2 - lat1
    d_lon = lon2 - lon1

    # Calculate the distance using the Haversine formula
    a = math.sin(d_lat/2) * math.sin(d_lat/2) + math.cos(lat1) * math.cos(lat2) * math.sin(d_lon/2) * math.sin(d_lon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    distance = radius * c

    return distance

In [ ]:
print(distance_between_coordinates(40.719810,-74.002581,40.735981, -74.029309))

In [ ]:
#build an alternative temporal graph
#two pois are connected if users go on average to them in the seme timeframe

df['localTime'] = pd.to_datetime(df['UTC time']) + pd.to_timedelta(df['Timezone'], unit = 'm')

df['localDayofWeek'] = df.localTime.dt.dayofweek
df['localHour'] = df.localTime.dt.hour






In [ ]:
venuesdf = df[['Venue ID','Latitude','Longitude','Venue category ID']].drop_duplicates(subset=['Venue ID'])
numvenues = len(venuesdf)
print(venuesdf)


In [ ]:
!pip3 install torch --index-url https://download.pytorch.org/whl/${CUDA}
import torch
print(torch.__version__)
print(torch.version.cuda)
!pip install torch-geometric \
  torch-sparse \
  torch-scatter \
  torch-cluster \
  -f https://pytorch-geometric.com/whl/torch-2.3.0+cu121.html

In [ ]:
import networkx as nx
venuesdict = venuesdf.to_dict('records')

i = 0
nodeIds = {}
node_attr = []

for el in venuesdict:
  #print(el['Venue ID'])
  nodeIds[el['Venue ID']] = i
  node_attr.append([float(el['Venue category ID'])])
  i+=1

print(len(node_attr))


In [ ]:
#import torch
sequences = {}
count = {}

MINVISIT = 5


for _, row in df.iterrows():

  if row['User ID'] not in sequences:
    sequences[row['User ID']] = [[row['Venue ID'],row['localHour'],row['localDayofWeek']]]


  else:
    sequences[row['User ID']].append([row['Venue ID'],row['localHour'],row['localDayofWeek']])


  if row['Venue ID'] not in count:
    count[row['Venue ID']] = 1
  else:
    count[row['Venue ID']] += 1

print(sequences)
seq = []
next = []
to_remove = []

for el in sequences:
  entry = [el]
  for i in range(len(sequences[el][len(sequences[el])-1])):
    el2 = sequences[el][len(sequences[el])-1][i]
    #print(el2,i)
    if i == 0:
      el2 = nodeIds[el2]
    #print(el2,i)
    entry.append(el2)
  for i in range(len(sequences[el])-1):
    entry.append(nodeIds[sequences[el][i][0]])
  seq.append(torch.tensor(entry))

  target = torch.zeros(numvenues)
  target[sequences[el][len(sequences[el])-1][0]] = 1
  next.append(target)
  #print(count[sequences[el][len(sequences[el])-1][0]])
  if count[sequences[el][len(sequences[el])-1][0]] < MINVISIT:

    to_remove.append(sequences[el][len(sequences[el])-1][0])
    #seq.append(torch.tensor(entry))
    #next.append(target)

seq=torch.nn.utils.rnn.pad_sequence(seq,batch_first=True)
seq_dim = len(seq[0])-4
print(seq_dim)
#seq=torch.nn.utils.rnn.pad_sequence(seq)

print(len(to_remove))

#to_remove_new = []

#for el in to_remove:

#  el = nodeIds[el]
#  to_remove_new[]

print(seq)
print(len(next))



In [ ]:
df = df[~df['Venue ID'].isin(to_remove)]
venuesdf = venuesdf[~venuesdf['Venue ID'].isin(to_remove)]
venuesdict = venuesdf.to_dict('records')
print(df)
print(venuesdf)

In [ ]:
#fast spatial graph construction

import networkx as nx
venuesdict = venuesdf.to_dict('records')
#print(venuesdict)

#for el in venuesdict:
#  print((el,venuesdict[el]))
'''
G = nx.Graph()

distances = []

from tqdm import tqdm

for i in tqdm(range(len(venuesdict) - 1)):
    if not G.has_node(venuesdict[i]['Venue ID']):
        G.add_node(venuesdict[i]['Venue ID'], category=venuesdict[i]['Venue category ID'])

    for j in range(i + 1, len(venuesdict)):
        if not G.has_node(venuesdict[j]['Venue ID']):
            G.add_node(venuesdict[j]['Venue ID'], category=venuesdict[i]['Venue category ID'])

        if distance_between_coordinates(venuesdict[i]['Latitude'], venuesdict[i]['Longitude'],
                                        venuesdict[j]['Latitude'], venuesdict[j]['Longitude']) < 1:
            G.add_edge(venuesdict[i]['Venue ID'], venuesdict[j]['Venue ID'])
'''
from tqdm import tqdm
import torch
from torch_geometric.data import Data

edge_index = []
edges_attr = []

for i in tqdm(range(len(venuesdict) - 1)):

   #if venuesdict[i]['Venue ID'] not in node_attr:
   #  node_attr.append(venuesdict[i]['Venue ID'])

   for j in range(i + 1, len(venuesdict)):
    #  if venuesdict[j]['Venue ID'] not in node_attr:
    #     node_attr.append(venuesdict[j]['Venue ID'])


      if distance_between_coordinates(venuesdict[i]['Latitude'], venuesdict[i]['Longitude'],
                                    venuesdict[j]['Latitude'], venuesdict[j]['Longitude']) < 1:
        edge_index.append([nodeIds[venuesdict[i]['Venue ID']], nodeIds[venuesdict[j]['Venue ID']]])
        edges_attr.append([1])

spatG = Data(x=torch.tensor(node_attr), edge_index=torch.LongTensor(edge_index).t().contiguous(), edge_attr=torch.tensor(edges_attr))
print(spatG)


In [ ]:
import pickle

print(spatG)

#spatpG = G

pickle.dump(spatG,open('spatG.pickle','wb'))

#data = pickle.load(open('spatG.pickle','rb'))

In [ ]:
#nx.draw(G)
print(spatG)

In [ ]:

#sequantial relation graph
#build a graph that captures the sequence of actions made by the users

import networkx as nx

edge_index = []
edges_attr = []

usrloc = {}

#seqG = nx.Graph()

for _, row in df.iterrows():

  #if not seqG.has_node(row['Venue ID']):
  #  seqG.add_node(row['Venue ID'], category=row['Venue category ID'])

  if row['User ID'] not in usrloc:
    usrloc[row['User ID']] = row['Venue ID']

  else:
    #seqG.add_edge(usrloc[row['User ID']],row['Venue ID'])
    edge_index.append([nodeIds[usrloc[row['User ID']]], nodeIds[row['Venue ID']]])
    edges_attr.append(1)
    usrloc[row['User ID']] = row['Venue ID']


seqG = Data(x=torch.tensor(node_attr), edge_index=torch.tensor(edge_index).t().contiguous(), edge_attr=torch.tensor(edges_attr))
print(seqG)






In [ ]:
import pickle


print(seqG)

pickle.dump(seqG,open('tempG.pickle','wb'))

#data = pickle.load(open('tempG.pickle','rb'))

In [ ]:
import math
groupeddf = df.groupby(['Venue ID','Venue category ID'],as_index=False)
avghour = groupeddf['localHour'].mean()
avgday = groupeddf['localDayofWeek'].mean()

print(avghour)

timedf = pd.concat([avghour,avgday['localDayofWeek']],axis=1)



print(timedf)

hours = {}
week = {}

#week['workdays'] = []
#week['weekend'] = []


for _, row in timedf.iterrows():

  hour = math.floor(row['localHour'])
  day = math.floor(row['localDayofWeek'])

  if hour not in hours:
    #hours[hour] = [[row['Venue ID'],row['Venue category ID']]]
    hours[hour] = [[row['Venue ID'],row['Venue category ID']]]

  else:
    hours[hour].append([row['Venue ID'],row['Venue category ID']])

  #if 1 <= day <=4:
  #  week['workdays'].append([row['Venue ID'],row['Venue category ID']])
  #else:
  #  week['weekend'].append([row['Venue ID'],row['Venue category ID']])

  if day not in week:
    #hours[hour] = [[row['Venue ID'],row['Venue category ID']]]
    week[day] = [[row['Venue ID'],row['Venue category ID']]]

  else:
    week[day].append([row['Venue ID'],row['Venue category ID']])






In [ ]:
#dayG = nx.Graph()
#hourG = nx.Graph()

edge_index = []
edges_attr = []

for el in hours:
   for u in hours[el]:

    #if not hourG.has_node(u[0]):
    #  hourG.add_node(u[0], category=u[1])

    for v in hours[el]:

      if u != v:

        #if not hourG.has_node(u[0]):
        #  hourG.add_node(v[0], category=v[1])

        #hourG.add_edge(u[0],v[0])
        edge_index.append([nodeIds[u[0]], nodeIds[v[0]]])

hourG = Data(x=torch.tensor(node_attr), edge_index=torch.tensor(edge_index).t().contiguous(), edge_attr=torch.tensor(edges_attr))
print(hourG)




In [ ]:
pickle.dump(hourG,open('hourG.pickle','wb'))

#data = pickle.load(open('tempG.pickle','rb'))

In [ ]:
edge_index = []
edges_attr = []


for el in week:
   for u in week[el]:

    #if not dayG.has_node(u[0]):
    #  dayG.add_node(u[0], category=u[1])

    for v in week[el]:

      if u != v:

        #if not dayG.has_node(u[0]):
        #  dayG.add_node(v[0], category=v[1])

        #dayG.add_edge(u[0],v[0])
        edge_index.append([nodeIds[u[0]], nodeIds[v[0]]])

dayG = Data(x=torch.tensor(node_attr), edge_index=torch.tensor(edge_index).t().contiguous(), edge_attr=torch.tensor(edges_attr))
print(dayG)

In [ ]:
pickle.dump(dayG,open('dayG.pickle','wb'))

In [ ]:
!pip install pytorch-lightning
import pytorch_lightning as pl

In [ ]:


import matplotlib.pyplot as plt
import torch
from sklearn.manifold import TSNE
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping

from torch_geometric.datasets import Planetoid
from torch_geometric.nn import Node2Vec

# Lightning Module for Node2Vec model
class Node2VecModel(pl.LightningModule):
    def __init__(self, data, embedding_dim=32, walk_length=20, context_size=10, walks_per_node=10, num_negative_samples=1, p=1.0, q=1.0, lr=0.01):
        super().__init__()
        self.data = data
        self.model = Node2Vec(
            data.edge_index,
            embedding_dim=embedding_dim,
            walk_length=walk_length,
            context_size=context_size,
            walks_per_node=walks_per_node,
            num_negative_samples=num_negative_samples,
            p=p,
            q=q,
            sparse=True
        )
        self.lr = lr

    def forward(self):
        return self.model()

    def training_step(self, batch, batch_idx):
        pos_rw, neg_rw = batch
        loss = self.model.loss(pos_rw.to(self.device), neg_rw.to(self.device))
        self.log('train_loss', loss)
        return loss

    def configure_optimizers(self):
        return torch.optim.SparseAdam(self.parameters(), lr=self.lr)

    def test_step(self, batch, batch_idx):
        z = self.model()
        acc = self.model.test(
            train_z=z[self.data.train_mask],
            train_y=self.data.y[self.data.train_mask],
            test_z=z[self.data.test_mask],
            test_y=self.data.y[self.data.test_mask],
            max_iter=150
        )
        self.log('test_acc', acc)
        return acc




batch_size = 128
num_workers = 4
graph_embed = []

for el in [spatG,hourG,seqG,dayG]:

  model = Node2VecModel(data=el)

  loader = model.model.loader(batch_size=128, shuffle=True, num_workers=num_workers)

  trainer = pl.Trainer(
      max_epochs=1,
      callbacks=[
          ModelCheckpoint(monitor='train_loss'),
          EarlyStopping(monitor='train_loss', patience=10)
      ]
  )
  trainer.fit(model, loader)
  #trainer.test(model, loader)

  # Plotting points


  model.eval()
  z = model().detach()
  graph_embed.append(z)

  #z = TSNE(n_components=2).fit_transform(z)
  print(z.shape)


In [ ]:
from sklearn.model_selection import train_test_split

batchsize = 32





#train_x,test_x,train_y,test_y = train_test_split(seq,torch.stack(next),test_size=0.2,stratify=torch.stack(next))
train_x,test_x,train_y,test_y = train_test_split(seq,torch.stack(next),test_size=0.2)
#train_x,val_x,train_y,val_y = train_test_split(train_x,train_y,test_size=0.2,stratify=train_y)
train_x,val_x,train_y,val_y = train_test_split(train_x,train_y,test_size=0.2)

trainset = torch.utils.data.TensorDataset(train_x,train_y)
trainloader = torch.utils.data.DataLoader(trainset,batch_size=batchsize,shuffle=True)
print(len(trainloader))

valset = torch.utils.data.TensorDataset(val_x,val_y)
valloader = torch.utils.data.DataLoader(valset,batch_size=batchsize,shuffle=True)
print(len(valloader))

testset = torch.utils.data.TensorDataset(test_x,test_y)
testloader = torch.utils.data.DataLoader(testset,batch_size=batchsize,shuffle=True)
print(len(testloader))

In [ ]:
#import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
'''
class GCN(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, 64)
        self.conv2 = GCNConv(64, out_channels)

    def forward(self, x, edge_index):
        #print(x)
        #print("sono una nuova convoluzione")
        #print(edge_index)
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return x
'''
class Model(nn.Module):
    def __init__(self, input_dim, location_dim, sequence_dim,graph_embed):
        super(Model, self).__init__()

        self.graph_embed = graph_embed

        # User Embedding Layer
        self.user_embedding = nn.Embedding(num_embeddings=input_dim, embedding_dim=32)

        #Last location Embedding Layer
        self.lastLoc_embedding = nn.Embedding(num_embeddings=input_dim, embedding_dim=32)

        #Time and day Input linear

        self.timeday_embedding = nn.Linear(2,32)
        '''
        # Location Embedding Layers
        self.gcn1 = GCN(spatG.num_features, 64)
        self.gcn2 = GCN(seqG.num_features, 64)
        self.gcn3 = GCN(hourG.num_features, 64)
        self.gcn4 = GCN(dayG.num_features, 64)
        '''

        # Sequence Embedding Layer
        self.self_attn = nn.MultiheadAttention(embed_dim=sequence_dim, num_heads=1)

        #Expand sequence representation

        self.expand_sequence = nn.Linear(sequence_dim,32)

        #Reduce embed dimensionality

        #self.compress_embed = nn.Linear()

        # Multihead Attention Layer
        self.multihead_attn = nn.MultiheadAttention(embed_dim=32, num_heads=2)

        #print(len(spatG.x))

        # Classifier Layers
        self.linear1 = nn.Linear(int((4*32*(self.graph_embed[0].shape[0]+batchsize))/batchsize), 128)
        self.linear2 = nn.Linear(128, 64)
        self.linear3 = nn.Linear(64, numvenues)

    def forward(self, user, lastloc, lasthour, lastday, sequence):
        #print(lastday)
        #curr_batch = len(user)
        #print(len(user))


        user_embed = self.user_embedding(user.int())

        lastloc_embed = self.lastLoc_embedding(lastloc.int())

        timeday_embed = self.timeday_embedding(torch.stack([lasthour.float(), lastday.float()],dim=1))


        graph_embed = torch.cat(self.graph_embed)
        #print(graph_embed.shape)
        #nodes = spatG.x[lastloc]

        #print(nodes)

        #k = 1
        # Location Embeddings
        #nodes,edge_index1,mapping,_ =torch_geometric.utils.k_hop_subgraph(lastloc,k,spatG.edge_index)
        #print(len(nodes))

        #nodes = spatG.x[nodes]

        #print(nodes)

        #print(isinstance(spatG.edge_index, torch.Tensor))
        #print(spatG.edge_index)

        '''

        location_embed = self.gcn1(x=spatG.x, edge_index=spatG.edge_index)
        seq_embed = self.gcn2(x=seqG.x, edge_index=seqG.edge_index)
        hour_embed = self.gcn3(x=hourG.x,edge_index=hourG.edge_index)
        day_embed =  self.gcn4(x=dayG.x,edge_index=dayG.edge_index)
        '''
        #print(sequence)

        # Sequence Embedding
        sequence_embed, _ = self.self_attn(sequence.float(), sequence.float(), sequence.float())

        sequence_embed = self.expand_sequence(sequence_embed)

        '''
        print(user_embed.shape)
        #print(location_embed.shape)
        #print(seq_embed.shape)
        #print(hour_embed.shape)
        #print(day_embed.shape)
        print(sequence_embed.shape)
        print(lastloc_embed.shape)
        print(timeday_embed.shape)
        '''
        #user_embed = user_embed*sequence_embed

        # Concatenate all embeddings
        concat_embed = torch.cat([user_embed,lastloc_embed,graph_embed,timeday_embed,sequence_embed], dim=0)

        #concat_embed = concat_embed.view(batchsize,-1)


        #print(concat_embed.shape)
        #print(len(spatG.x))

        # Multihead Attention
        attn_output, _ = self.multihead_attn(concat_embed, concat_embed, concat_embed)

        #print(attn_output.shape)


        # Classifier
        out = F.relu(self.linear1(attn_output.view(batchsize,-1)))
        out = F.relu(self.linear2(out))
        out = self.linear3(out)

        return out

# Define Lightning model using PyTorch Lightning
import pytorch_lightning as pl

class LightningModel(pl.LightningModule):
    def __init__(self, input_dim, location_dim, sequence_dim,graph_embed):
        super(LightningModel, self).__init__()
        self.model = Model(input_dim, location_dim, sequence_dim,graph_embed)

    def forward(self, user, timestamp, location, sequence):
        return self.model(user, timestamp, location, sequence)

    def training_step(self, batch, batch_idx):

        x, y = batch
        #print(x.shape,y.shape)
        pad_dim = batchsize-x.shape[0]

        if pad_dim != 0:

         x = torch.cat([x,torch.zeros(pad_dim,x.shape[1])])
         y =torch.cat([y,torch.zeros(pad_dim,y.shape[1])])

        #print(x)
        y_pred = self.model(x[:,0],x[:,1],x[:,2],x[:,3],x[:,4:])
        loss = F.cross_entropy(torch.sigmoid(y_pred), y)
        print(loss)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
        return optimizer

# Example of training the model
input_dim = 10000
location_dim = 32
sequence_dim = seq_dim
#print(seq_dim)
model = LightningModel(input_dim, location_dim, sequence_dim,graph_embed)

trainer = pl.Trainer(max_epochs=10)
trainer.fit(model, trainloader)